# <font color='dimgrey'> RPC using RabbitMQ </font>


## <font color='grey'> What's an RPC </font><br>
<font color='grey'>
It's a remote procedural call when made, the client waits for the response from the remote server while blocking the client  </font>

## <font color='grey'> How to make an rpc using rabbitmq </font><br>

<font color='grey'>
It can be summarized in the following steps  <br>
    1. A client publishes a message to the rpc queue with a callback queue name in the request and a correlation_id<br>
    2. Server is listening to the request. It process the call, publishes a message to the callback queue it recieved from the first step<br>
    3. Client is listening to this queue, it checks the response and the correlation_id with it.<br>
    4. Client compares the correlation_id with the correlation_id it send in the first step, if matches saves the response/answer
</font>

<img src="https://www.rabbitmq.com/img/tutorials/python-six.png"/>